In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile, os, urllib.request, glob, math, shutil, sys
import torchvision
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, precision_recall_fscore_support, roc_auc_score, accuracy_score
import sklearn.metrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import PIL
import PIL.Image

sys.path.append('drive/MyDrive')
import modelArchs

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Κάνω load τα μοντέλα.

In [ ]:
#load resnet34
resnet_derm = modelArchs.get_Resnet(8)
resnet_derm.to(device)
resnet_derm.load_state_dict(torch.load('drive/MyDrive/resnet34_derm.pt')['model'])
resnet_derm.eval()

#load mobilenet
mobilenet_derm = modelArchs.get_MobileNet(8)
mobilenet_derm.to(device)
mobilenet_derm.load_state_dict(torch.load('drive/MyDrive/mobilenet_derm.pt')['model'])
mobilenet_derm.eval()

#load googlenet
googlenet_derm = modelArchs.get_GoogleNet(8)
googlenet_derm.to(device)
googlenet_derm.load_state_dict(torch.load('drive/MyDrive/googlenet_derm.pt')['model'])
googlenet_derm.eval()

#load efficient net
efficient_net_derm = modelArchs.get_EfficientNet(8)
efficient_net_derm.to(device)
efficient_net_derm.load_state_dict(torch.load('drive/MyDrive/efficient_net_derm.pt')['model'])
efficient_net_derm.eval()

#load mobilenet + sa
mobilenet_sa_derm = modelArchs.get_MobileNetSa(8)
mobilenet_sa_derm.to(device)
mobilenet_sa_derm.load_state_dict(torch.load('drive/MyDrive/mobilenet_sa_derm.pt')['model'])
mobilenet_sa_derm.eval()

Οι εικόνες είναι αποθηκευμένες στο drive σε zip.

Υπάρχουν 16 zip files, ένα για κάθε κλάση από το train και test dataset.

Κάνω extract τις εικόνες.

In [ ]:
os.makedirs('test_imgs')

prefix = 'drive/MyDrive/'

for i in ['test_imgs']:
  for j in ['MEL','NV','BCC','AK','BKL','DF','VASC','SCC']:

    name = i + '_' + j
    src_zip = prefix + name + '.zip'
    dest_zip = i + '/' + j

    with zipfile.ZipFile(src_zip, 'r') as zip_ref:
      zip_ref.extractall(dest_zip)

    print("Finished with: ", name)

Χρήση των transformation που χρησιμοποιήθηκαν στο testing των μοντέλων.

In [ ]:
transforms_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_dataset = pd.read_csv('drive/MyDrive/test_dataset.csv')
val_dataset = datasets.ImageFolder('test_imgs', transforms_val)
len_val_dataset = len(val_dataset)

print("Length of val dataset: ", len_val_dataset)
batch_len = 64
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_len, shuffle=False)

Υπολογισμός των metrics, χρησιμοποιώντας average ensembling με τα μοντέλα που
έκανα train.

In [ ]:
def get_accuracy(logit, target, batch_size):

  correct_classification = 0

  for i in range(batch_size):
    if logit[i] == target[i]:
      correct_classification += 1

  return (correct_classification / batch_size) * 100

def get_predictions(models_out):

  num_models = len(models_out)

  #calculate softmax
  model_probs = []
  for i in range(num_models):
    temp_prob = torch.nn.functional.softmax(models_out[i],dim = 1)
    model_probs.append(temp_prob)

  stacked_outs = torch.stack(model_probs, 0)
  mean_outs = torch.mean(stacked_outs, 0)

  probs = torch.nn.functional.softmax(mean_outs, dim = 1).tolist()
  predictions = torch.max(mean_outs,1)[1].tolist()

  return probs, predictions

def test_ensemble(try_models, test_dataloader):

  num_models = len(try_models)

  test_acc = 0.0
  pred_list = []
  label_list = []
  output_probs = []
  metrics = []

  for i, (inputs, labels) in enumerate(test_dataloader,1):

    label_list.extend(labels.detach().tolist())

    inputs = inputs.to(device)
    labels = labels.to(device)

    models_out = []
    for j in range(num_models):
      temp_out = try_models[j](inputs).detach().cpu()
      models_out.append(temp_out)

    probs, preds = get_predictions(models_out)

    output_probs.extend(probs)
    pred_list.extend(preds)

  conf_matrix = confusion_matrix(label_list , pred_list)

  metrics.append(accuracy_score(label_list, pred_list))
  metrics.extend(precision_recall_fscore_support(label_list,pred_list, average = 'macro'))
  metrics.append(roc_auc_score(label_list,output_probs, average = 'macro', multi_class = 'ovr'))
  metrics.extend(precision_recall_fscore_support(label_list,pred_list, average = 'weighted'))
  metrics.append(roc_auc_score(label_list,output_probs, average = 'weighted', multi_class = 'ovr'))

  return metrics, conf_matrix

Υπολογισμός των αποτελεσμάτων με ensemble model.

In [ ]:
#apotelesmata test set
models_try = [resnet_derm, mobilenet_derm, googlenet_derm, mobilenet_sa_derm,  efficient_net_derm]
metrics, conf_matrix = test_ensemble(models_try,val_dataloader)
print(metrics)
print(conf_matrix)